<a href="https://colab.research.google.com/github/CelikAbdullah/deep-learning-notebooks/blob/main/Computer%20Vision/models/Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow import keras

# Inception v1

## The stem component

In [2]:
def stem(inputs):
  '''
  Build the stem component of Inception v1.
  It extracts the coarse-level features of the input.

  input: the input image
  '''

  # apply ZeroPadding so that the size of the input is preserved
  x = keras.layers.ZeroPadding2D(padding=3)(inputs)

  # apply a strided 7x7 convolution to extract features
  # it will generate 64 output feature maps
  x = keras.layers.Conv2D(filters=64,
                          kernel_size=7,
                          strides=2,
                          padding="valid",
                          activation="relu",
                          kernel_initializer="glorot_uniform"
                          )(x)

  # again, apply ZeroPadding
  x = keras.layers.ZeroPadding2D(padding=1)(x)

  # apply strided max pooling with a 3x3 window
  # the size of each feature map will be reduced by 75%
  x = keras.layers.MaxPooling2D(pool_size=3, strides=2)(x)

  # apply a 1x1 convolution
  x = keras.layers.Conv2D(filters = 64,
                          kernel_size=1,
                          padding="same",
                          activation="relu",
                          kernel_initializer="glorot_uniform")(x)

  # zero-padding
  x = keras.layers.ZeroPadding2D(padding=1)(x)

  # a 3x3 convolution; it will output 192 feature maps
  x = keras.layers.Conv2D(filters=192,
                          kernel_size=3,
                          strides=1,
                          padding="valid",
                          activation="relu",
                          kernel_initializer="glorot_uniform"
                          )(x)

  # zero-padding
  x = keras.layers.ZeroPadding2D(padding=1)(x)

  # apply strided max pooling with a 3x3 window
  # the size of each feature map will be reduced by 75%
  x = keras.layers.MaxPooling2D(pool_size=3, strides=2)(x)


  return x

## The learner component

### Auxiliary Classifier

In [3]:
def auxiliary(x, classes):
  '''
  Build the auxiliary classifier.

  x       : input to the auxiliary classifier
  classes : number of output classes
  '''

  # strided average pooling operation with a 5x5 window
  x = keras.layers.AveragePooling2D(pool_size=5, strides=3)(x)

  # a 1x1 convolution which will output 128 feature maps
  x = keras.layers.Conv2D(filters=128,
                          kernel_size=1,
                          padding="same",
                          activation="relu",
                          kernel_initializer="glorot_uniform"
                          )(x)

  # flatten the feature maps before passing them to the Dense layer
  x = keras.layers.Flatten()(x)

  # a fully connected layer with 1024 neurons
  x = keras.layers.Dense(units=1024,
                         activation="relu",
                         kernel_initializer="glorot_uniform")(x)

  # dropout layer for regularization
  x = keras.layers.Dropout(rate=0.7)(x)

  # softmax layer to output probabilities for each class
  output = keras.layers.Dense(units=classes,
                              activation="softmax",
                              kernel_initializer="glorot_uniform")(x)


  return output

### Inception Block

In [4]:
def inception_block(input, filters_1x1, filters_3x3, filters_5x5, filter_pool):
  '''
  Build the inception block/module.

  input : input to the inception block/module
  filters_1x1 : nr. of filters for the 1x1 conv branch of the inception module
  filters_3x3 : nr. of filters for the 3x3 conv branch of the inception model
  filters_5x5 : nr. of filters for the 5x5 conv branch of the inception module
  filters_pool: nr. of filters for the max pooling branch of the inception module
  '''

  # create the 1x1 conv branch
  branch_1x1 = keras.layers.Conv2D(filters=filters_1x1[0],
                                   kernel_size=1,
                                   strides=1,
                                   padding="same",
                                   activation='relu',
                                   kernel_initializer='glorot_uniform')(input)

  # create the 3x3 conv branch with the 1x1 bottleneck convolution that performs dimensionality reduction
  branch_3x3 = keras.layers.Conv2D(filters=filters_3x3[0],
                                   kernel_size=1,
                                   strides=1,
                                   padding="same",
                                   activation='relu',
                                   kernel_initializer='glorot_uniform')(input)

  branch_3x3 = keras.layers.ZeroPadding2D(padding=1)(branch_3x3)
  branch_3x3 = keras.layers.Conv2D(filters=filters_3x3[1],
                                   kernel_size=3,
                                   strides=1,
                                   padding="valid",
                                   activation='relu',
                                   kernel_initializer='glorot_uniform')(branch_3x3)

  # create the 5x5 conv branch with the 1x1 bottleneck convolution that performs dimensionality reduction
  branch_5x5 = keras.layers.Conv2D(filters=filters_5x5[0],
                                   kernel_size=1,
                                   strides=1,
                                   padding="same",
                                   activation='relu',
                                   kernel_initializer='glorot_uniform')(input)

  branch_5x5 = keras.layers.ZeroPadding2D(padding=1)(branch_5x5)
  branch_5x5 = keras.layers.Conv2D(filters=filters_5x5[1],
                                   kernel_size=3,
                                   strides=1,
                                   padding="valid",
                                   activation='relu',
                                   kernel_initializer='glorot_uniform')(branch_5x5)

  # create the max pooling branch with 1x1 linear projection for dimensionality expansion
  branch_pool = keras.layers.MaxPooling2D(pool_size=3,
                                          strides=1,
                                          padding="same")(input)

  branch_pool = keras.layers.Conv2D(filters=filter_pool[0],
                                    kernel_size=1,
                                    strides=1,
                                    padding='same',
                                    activation='relu',
                                    kernel_initializer='glorot_uniform')(branch_pool)


  # Concatenate the output feature maps of the branches
  x = keras.layers.Concatenate()([branch_1x1, branch_3x3, branch_5x5, branch_pool])

  return x


### Inception Group

In [5]:
def group(x, blocks, pooling=True, classes=1000):
  '''
  Build an Inception group.

  x       : input to the inception group
  blocks  : list of tuples containing nr. of filters for each block in the group
  pooling : flag indicating whether to end the group with max pooling
  classes : nr. of classes for auxiliary classifier
  '''

  # list to save the auxiliary outputs
  aux = []

  # build the inception blocks
  for block in blocks:

    # adding the auxiliary classifier
    if block is None:
      # save the auxiliary output
      aux.append(auxiliary(x,classes))

    else:
       # create inception block
       x = inception_block(x,
                           block[0],
                           block[1],
                           block[2],
                           block[3])


  # if pooling is True
  if pooling:
    # zero-padding
    x = keras.layers.ZeroPadding2D(padding=(1,1))(x)

    # max pooling
    x = keras.layers.MaxPooling2D(pool_size=3, strides=2)(x)


  return x, aux

Now, we can finally build the learner component of Inception v1:

In [6]:
def learner(x, classes):
  '''
  Build the learner component.

  x       : input to the learner
  classes : nr. of classes
  '''

  # a list to store outputs from the auxiliary classifiers
  aux = []

  # set the block specs for group 3
  blocks = [((64,),  (96,128),   (16, 32), (32,)),
            ((128,), (128, 192), (32, 96), (64,))]

  # build group 3
  x, o = group(x, blocks)
  aux += o

  # set block spec for group 4
  blocks = [((192,),  (96, 208), (16, 48), (64,)),
            None,
            ((160,), (112, 224), (24, 64), (64,)),
            ((128,), (128, 256), (24, 64), (64,)),
            ((112,), (144, 288), (32, 64), (64,)),
            None,
            ((256,), (160, 320), (32, 128), (128,))]

  # build group 4
  x, o = group(x, blocks, classes=classes)
  aux += o

  # set block specs for group 5
  blocks = [((256,), (160, 320), (32, 128), (128,)),
            ((384,), (192, 384), (48, 128), (128,))]
  # build group 5
  x, o = group(x, blocks, pooling=False)
  aux += o


  return x, aux

## The task component

In [7]:
def task(x, classes, dropout=0.4):
  '''
  Build the task component.

  x       : input to the task component
  classes : nr. of output classes
  dropout : dropout rate
  '''

  # a 7x7 average pooling
  x = keras.layers.AveragePooling2D(pool_size=7)(x)

  # flatten the output from the avg pooling
  x = keras.layers.Flatten()(x)

  # dropout for regularization
  x = keras.layers.Dropout(rate=dropout)(x)

  # softmax layer
  outputs = keras.layers.Dense(units=classes,
                               activation="softmax",
                               kernel_initializer="glorot_uniform")(x)

  return outputs

## Inception model v1

In [8]:
def build_model(shape=(224, 224, 3), dropout=0.4):

  # define inputs
  inputs = keras.layers.Input(shape=shape)

  # the stem component
  x = stem(inputs)

  # the learner component
  x, aux = learner(x, 1000)

  # the task component
  outputs = task(x, 1000, dropout)

  # build the model
  model = keras.Model(inputs=inputs,
                      outputs=[outputs] + aux,
                      name="inception_v1")

  return model

In [9]:
# create the inception model (v1)
inception_model_v1 = build_model()

# print a summary
inception_model_v1.summary()

Model: "inception_v1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 64)         9472      ['zero_padding2d[0][0]']      
                                                                                                  
 zero_padding2d_1 (ZeroPadd  (None, 114, 114, 64)         0         ['conv2d[0][0]']   

# Inception v2

## The stem component

In [10]:
def stem(inputs):
  '''
  Build the stem component of Inception v1.
  It extracts the coarse-level features of the input.

  input: the input image
  '''

  # apply ZeroPadding so that the size of the input is preserved
  x = keras.layers.ZeroPadding2D(padding=3)(inputs)

  # apply a strided 7x7 convolution to extract features
  # it will generate 64 output feature maps
  x = keras.layers.Conv2D(filters=64,
                          kernel_size=7,
                          strides=2,
                          padding="valid",
                          use_bias=False,
                          kernel_initializer="glorot_uniform"
                          )(x)

  # a BatchNormalization layer followed by a ReLu activation layer
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # again, apply ZeroPadding
  x = keras.layers.ZeroPadding2D(padding=1)(x)

  # apply strided max pooling with a 3x3 window
  # the size of each feature map will be reduced by 75%
  x = keras.layers.MaxPooling2D(pool_size=3, strides=2)(x)

  # apply a 1x1 convolution
  x = keras.layers.Conv2D(filters = 64,
                          kernel_size=1,
                          padding="same",
                          use_bias = False,
                          kernel_initializer="glorot_uniform")(x)

  # a BatchNormalization layer followed by a ReLu activation layer
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # zero-padding
  x = keras.layers.ZeroPadding2D(padding=1)(x)

  # a 3x3 convolution; it will output 192 feature maps
  x = keras.layers.Conv2D(filters=192,
                          kernel_size=3,
                          strides=1,
                          padding="valid",
                          use_bias=False,
                          kernel_initializer="glorot_uniform"
                          )(x)

  # a BatchNormalization layer followed by a ReLu activation layer
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # zero-padding
  x = keras.layers.ZeroPadding2D(padding=1)(x)

  # apply strided max pooling with a 3x3 window
  # the size of each feature map will be reduced by 75%
  x = keras.layers.MaxPooling2D(pool_size=3, strides=2)(x)


  return x

## The learner component

### Auxiliary Classifier

In [11]:
def auxiliary(x, classes):
  '''
  Build the auxiliary classifier.

  x       : input to the auxiliary classifier
  classes : number of output classes
  '''

  # strided average pooling operation with a 5x5 window
  x = keras.layers.AveragePooling2D(pool_size=5, strides=3)(x)

  # a 1x1 convolution which will output 128 feature maps
  x = keras.layers.Conv2D(filters=128,
                          kernel_size=1,
                          padding="same",
                          use_bias=False,
                          kernel_initializer="glorot_uniform"
                          )(x)

  # a BatchNormalization layer followed by a ReLU action layer
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # flatten the feature maps before passing them to the Dense layer
  x = keras.layers.Flatten()(x)

  # a fully connected layer with 1024 neurons
  x = keras.layers.Dense(units=1024,
                         activation="relu",
                         kernel_initializer="glorot_uniform")(x)

  # dropout layer for regularization
  x = keras.layers.Dropout(rate=0.7)(x)

  # softmax layer to output probabilities for each class
  output = keras.layers.Dense(units=classes,
                              activation="softmax",
                              kernel_initializer="glorot_uniform")(x)


  return output

### Inception Block

In [12]:
def inception_block(input, filters_1x1, filters_3x3, filters_5x5, filter_pool):
  '''
  Build the inception block/module.

  input : input to the inception block/module
  filters_1x1 : nr. of filters for the 1x1 conv branch of the inception module
  filters_3x3 : nr. of filters for the 3x3 conv branch of the inception model
  filters_5x5 : nr. of filters for the 5x5 conv branch of the inception module
  filters_pool: nr. of filters for the max pooling branch of the inception module
  '''

  # create the 1x1 conv branch with a BatchNormalization followed by a ReLU activation
  branch_1x1 = keras.layers.Conv2D(filters=filters_1x1[0],
                                   kernel_size=1,
                                   strides=1,
                                   padding="same",
                                   use_bias=False,
                                   kernel_initializer='glorot_uniform')(input)

  branch_1x1 = keras.layers.BatchNormalization()(branch_1x1)
  branch_1x1 = keras.layers.ReLU()(branch_1x1)

  # create the 3x3 conv branch with the 1x1 bottleneck convolution that performs dimensionality reduction
  branch_3x3 = keras.layers.Conv2D(filters=filters_3x3[0],
                                   kernel_size=1,
                                   strides=1,
                                   padding="same",
                                   use_bias=False,
                                   kernel_initializer='glorot_uniform')(input)

  # a BatchNormalization layer followed by a ReLU action layer
  branch_3x3 = keras.layers.BatchNormalization()(branch_3x3)
  branch_3x3 = keras.layers.ReLU()(branch_3x3)

  # zero-padding
  branch_3x3 = keras.layers.ZeroPadding2D(padding=1)(branch_3x3)

  # a 3x3 conv layer for feature extraction
  branch_3x3 = keras.layers.Conv2D(filters=filters_3x3[1],
                                   kernel_size=3,
                                   strides=1,
                                   padding="valid",
                                   use_bias=False,
                                   kernel_initializer='glorot_uniform')(branch_3x3)

  # a BatchNormalization layer followed by a ReLU activation layer
  branch_3x3 = keras.layers.BatchNormalization()(branch_3x3)
  branch_3x3 = keras.layers.ReLU()(branch_3x3)


  # create the 5x5 conv branch with the 1x1 bottleneck convolution that performs dimensionality reduction
  branch_5x5 = keras.layers.Conv2D(filters=filters_5x5[0],
                                   kernel_size=1,
                                   strides=1,
                                   padding="same",
                                   use_bias=False,
                                   kernel_initializer='glorot_uniform')(input)

  # a BatchNormalization layer followed by a ReLU activation layer
  branch_5x5 = keras.layers.BatchNormalization()(branch_5x5)
  branch_5x5 = keras.layers.ReLU()(branch_5x5)

  # zero-padding
  branch_5x5 = keras.layers.ZeroPadding2D(padding=1)(branch_5x5)

  # a 3x3 convolution layer
  branch_5x5 = keras.layers.Conv2D(filters=filters_5x5[1],
                                   kernel_size=3,
                                   strides=1,
                                   padding="valid",
                                   use_bias=False,
                                   kernel_initializer='glorot_uniform')(branch_5x5)

  # a BatchNormalization layer followed by a ReLU activation layer
  branch_5x5 = keras.layers.BatchNormalization()(branch_5x5)
  branch_5x5 = keras.layers.ReLU()(branch_5x5)


  # create the max pooling branch with 1x1 linear projection for dimensionality expansion
  branch_pool = keras.layers.MaxPooling2D(pool_size=3,
                                          strides=1,
                                          padding="same")(input)

  # a 1x1 convolution layer
  branch_pool = keras.layers.Conv2D(filters=filter_pool[0],
                                    kernel_size=1,
                                    strides=1,
                                    padding='same',
                                    use_bias=False,
                                    kernel_initializer='glorot_uniform')(branch_pool)
  # a BatchNormalization layer followed by a ReLU activation layer
  branch_pool = keras.layers.BatchNormalization()(branch_pool)
  branch_pool = keras.layers.ReLU()(branch_pool)

  # Concatenate the output feature maps of the branches
  x = keras.layers.Concatenate()([branch_1x1, branch_3x3, branch_5x5, branch_pool])

  return x


### Inception Group

In [13]:
def group(x, blocks, pooling=True, classes=1000):
  '''
  Build an Inception group.

  x       : input to the inception group
  blocks  : list of tuples containing nr. of filters for each block in the group
  pooling : flag indicating whether to end the group with max pooling
  classes : nr. of classes for auxiliary classifier
  '''

  # list to save the auxiliary outputs
  aux = []

  # build the inception blocks
  for block in blocks:

    # adding the auxiliary classifier
    if block is None:
      # save the auxiliary output
      aux.append(auxiliary(x,classes))

    else:
       # create inception block
       x = inception_block(x,
                           block[0],
                           block[1],
                           block[2],
                           block[3])


  # if pooling is True
  if pooling:
    # zero-padding
    x = keras.layers.ZeroPadding2D(padding=(1,1))(x)

    # max pooling
    x = keras.layers.MaxPooling2D(pool_size=3, strides=2)(x)


  return x, aux

### Putting it all together

Finally, we can build the learner component as follows:

In [14]:
def learner(x, classes):
  """ Build the learner component.

      x        : input to the learner
      classes: number of output classes
  """

  # a list to keep track of the auxiliary classifiers
  aux = []

  # the block specs for group 3
  blocks = [((64,),  (96,128),   (16, 32), (32,)),
              ((128,), (128, 192), (32, 96), (64,))]
  # build group 3
  x, o = group(x, blocks)
  aux += o

  # the block specs for group 4
  blocks = [((192,),  (96, 208), (16, 48), (64,)),
            None,
            ((160,), (112, 224), (24, 64), (64,)),
            ((128,), (128, 256), (24, 64), (64,)),
            ((112,), (144, 288), (32, 64), (64,)),
            None,
            ((256,), (160, 320), (32, 128), (128,))]
  # build group 4
  x, o = group(x, blocks, classes=classes)
  aux += o

  # the block specs for group 5
  blocks = [((256,), (160, 320), (32, 128), (128,)),
            ((384,), (192, 384), (48, 128), (128,))]
  # build group 5
  x, o = group(x, blocks, pooling=False)
  aux += o


  return x, aux

## The task component

In [15]:
def task(x, classes, dropout=0.4):
  '''
  Build the task component.

  x       : input to the task component
  classes : nr. of output classes
  dropout : dropout rate
  '''

  # a 7x7 average pooling
  x = keras.layers.AveragePooling2D(pool_size=7)(x)

  # flatten the output from the avg pooling
  x = keras.layers.Flatten()(x)

  # dropout for regularization
  x = keras.layers.Dropout(rate=dropout)(x)

  # softmax layer
  outputs = keras.layers.Dense(units=classes,
                               activation="softmax",
                               kernel_initializer="glorot_uniform")(x)

  return outputs

## Inception model v2

In [16]:
def build_model(shape=(224, 224, 3), dropout=0.4):

  # define inputs
  inputs = keras.layers.Input(shape=shape)

  # the stem component
  x = stem(inputs)

  # the learner component
  x, aux = learner(x, 1000)

  # the task component
  outputs = task(x, 1000, dropout)

  # build the model
  model = keras.Model(inputs=inputs,
                      outputs=[outputs] + aux,
                      name="inception_v2")

  return model

In [17]:
# create the inception model (v2)
inception_model_v2 = build_model()

# print a summary
inception_model_v2.summary()

Model: "inception_v2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d_24 (ZeroPad  (None, 230, 230, 3)          0         ['input_2[0][0]']             
 ding2D)                                                                                          
                                                                                                  
 conv2d_59 (Conv2D)          (None, 112, 112, 64)         9408      ['zero_padding2d_24[0][0]']   
                                                                                                  
 batch_normalization (Batch  (None, 112, 112, 64)         256       ['conv2d_59[0][0]']

# Inception v3

## The stem component

In [18]:
def stem(inputs):
  '''
  Construct the stem.
  The 7x7 coarse filter of v1 will be factorized into three 3x3 conv layers.
  The first conv layer is strided.
  '''
  # strided 3x3 conv layer
  x = keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, padding="valid", use_bias=False, kernel_initializer="glorot_uniform")(inputs)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)
  # 2nd 3x3 conv layer
  x = keras.layers.Conv2D(32, (3, 3), strides=(1, 1), padding='valid', use_bias=False, kernel_initializer='glorot_uniform')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)
  # 3rd 3x3 conv layer
  x = keras.layers.Conv2D(64, (3, 3), strides=(1, 1), padding='same', use_bias=False, kernel_initializer='glorot_uniform')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)
  # reduce pooled feature maps by 75%
  x = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
  # a 1x1 conv layer for dimensionality reduction
  x = keras.layers.Conv2D(80, (1, 1), strides=(1, 1), padding='valid', use_bias=False, kernel_initializer='glorot_uniform')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)
  # a 3x3 conv layer for dimensionality expansion
  x = keras.layers.Conv2D(192, (3, 3), strides=(1, 1), padding='valid', use_bias=False, kernel_initializer='glorot_uniform')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)
  # reduce pooled feature maps by 75%
  x = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

  return x


## The learner component

### Auxiliary Classifier

In [19]:
def auxiliary(x, classes):
  """ Build the auxiliary classier
      x         : input to the auxiliary classifier
      classes   : number of output classes
  """
  x = keras.layers.AveragePooling2D(pool_size=5, strides=(3, 3))(x)
  x = keras.layers.Conv2D(filters=128, kernel_size=1, strides=(1, 1), use_bias=False, kernel_initializer='glorot_uniform')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)
  x = keras.layers.Conv2D(filters=768, kernel_size=x.shape[1:3].as_list(), strides=(1, 1), use_bias=False, kernel_initializer='glorot_uniform')(x)
  x = keras.layers.Flatten()(x)

  output = keras.layers.Dense(units=classes, activation='softmax', kernel_initializer='glorot_uniform')(x)

  return output

### Inception Group

In [20]:
def group(x, blocks, inception=None, reduction=None, classes=0):
  """
  Build the Inception group.

  x         : input into the group
  blocks    : filters for each block in the group
  inception : type of inception block
  reduction : whether to end the group with grid reduction
  classes   : nr. of classes for auxiliary classifier
  """

  # list of auxiliary outputs
  auxiliary_outputs = []

  # build the inception blocks/modules
  for block in blocks:
    x = inception(x, block[0], block[1], block[2], block[3])

  # add the auxiliary classifier
  if classes:
    auxiliary_outputs.append(auxiliary(x, classes))

  # add grid reduction
  if reduction:
    x = reduction(x)

  return x, auxiliary_outputs

### Inception Block A

In [21]:
def inception_block_A(x, filters_1x1, filters_3x3, filters_5x5, filters_pool):
  """
  Build an Inception block 35x35.

  x             : input to the block
  filters_1x1   : filters for 1x1 branch
  filters_3x3   : filters for 3x3 branch
  filters_5x5   : filters for 5x5 branch
  filters_pool  : filters for pooling branch
  """

  # building the 1x1 branch
  branch_1x1 = keras.layers.Conv2D(filters=filters_1x1[0], kernel_size=1, strides=1, padding="same", use_bias=False, kernel_initializer="glorot_uniform")(x)
  branch_1x1 = keras.layers.BatchNormalization()(branch_1x1)
  branch_1x1 = keras.layers.ReLU()(branch_1x1)

  # build a double 3x3 branch with a 1x1 conv layer for dimensionality reduction
  branch_3x3 = keras.layers.Conv2D(filters=filters_3x3[0], kernel_size=1, strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(x)
  branch_3x3 = keras.layers.BatchNormalization()(branch_3x3)
  branch_3x3 = keras.layers.ReLU()(branch_3x3)

  # a 3x3 conv layer
  branch_3x3 = keras.layers.Conv2D(filters=filters_3x3[1], kernel_size=3, strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_3x3)
  branch_3x3 = keras.layers.BatchNormalization()(branch_3x3)
  branch_3x3 = keras.layers.ReLU()(branch_3x3)

  # a 3x3 conv layer
  branch_3x3 = keras.layers.Conv2D(filters=filters_3x3[1], kernel_size=3, strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_3x3)
  branch_3x3 = keras.layers.BatchNormalization()(branch_3x3)
  branch_3x3 = keras.layers.ReLU()(branch_3x3)

  # a 5x5 branch with a 1x1 conv layer for dimensionality reduction
  branch_5x5 = keras.layers.Conv2D(filters=filters_5x5[0], kernel_size=1, strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(x)
  branch_5x5 = keras.layers.BatchNormalization()(branch_5x5)
  branch_5x5 = keras.layers.ReLU()(branch_5x5)
  branch_5x5 = keras.layers.Conv2D(filters=filters_5x5[1], kernel_size=(3, 3), strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_5x5)
  branch_5x5 = keras.layers.BatchNormalization()(branch_5x5)
  branch_5x5 = keras.layers.ReLU()(branch_5x5)

  # a 1x1 pooling branch
  branch_pool = keras.layers.AveragePooling2D(pool_size=3, strides=1, padding='same')(x)
  branch_pool = keras.layers.Conv2D(filters=filters_pool[0], kernel_size=1, strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_pool)
  branch_pool = keras.layers.BatchNormalization()(branch_pool)
  branch_pool = keras.layers.ReLU()(branch_pool)

  # Concatenate the outputs (filters) of the branches
  x = keras.layers.Concatenate()([branch_1x1, branch_3x3, branch_5x5, branch_pool])

  return x

### Inception Block B

In [22]:
def inception_block_B(x, filters_1x1, filters_7x7, filters_7x7_double, filters_pool):
  """ Build an Inception block 17x17.

      x                 : input to the block
      filters_1x1       : filters for 1x1 branch
      filters_7x7       : filters for 7x7 factorized asn 1x7, 7x1 branch
      filters_7x7_double: filters for double 7x7 factorized as 1x7, 7x1 branch
      filters_pool      : filters for pooling branch
  """
  # build the 1x1 branch
  branch_1x1 = keras.layers.Conv2D(filters=filters_1x1[0], kernel_size=1, strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(x)
  branch_1x1 = keras.layers.BatchNormalization()(branch_1x1)
  branch_1x1 = keras.layers.ReLU()(branch_1x1)

  # build the 7x7 branch with a 1x1 conv layer for dimensionality reduction
  branch_7x7 = keras.layers.Conv2D(filters=filters_7x7[0], kernel_size=1, strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(x)
  branch_7x7 = keras.layers.BatchNormalization()(branch_7x7)
  branch_7x7 = keras.layers.ReLU()(branch_7x7)
  # a 1x7 conv layer
  branch_7x7 = keras.layers.Conv2D(filters=filters_7x7[1], kernel_size=(1, 7), strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_7x7)
  branch_7x7 = keras.layers.BatchNormalization()(branch_7x7)
  branch_7x7 = keras.layers.ReLU()(branch_7x7)
  # a 7x1 conv layer
  branch_7x7 = keras.layers.Conv2D(filters=filters_7x7[2], kernel_size=(7, 1), strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_7x7)
  branch_7x7 = keras.layers.BatchNormalization()(branch_7x7)
  branch_7x7 = keras.layers.ReLU()(branch_7x7)

  # build the double 7x7 branch with a 1x1 conv layer for dimensionality reduction
  branch_7x7_double = keras.layers.Conv2D(filters=filters_7x7_double[0], kernel_size=1, strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(x)
  branch_7x7_double = keras.layers.BatchNormalization()(branch_7x7_double)
  branch_7x7_double = keras.layers.ReLU()(branch_7x7_double)
  # a 1x7 conv layer
  branch_7x7_double = keras.layers.Conv2D(filters=filters_7x7_double[1], kernel_size=(1, 7), strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_7x7_double)
  branch_7x7_double = keras.layers.BatchNormalization()(branch_7x7_double)
  branch_7x7_double = keras.layers.ReLU()(branch_7x7_double)
  # a 7x1 conv layer
  branch_7x7_double = keras.layers.Conv2D(filters=filters_7x7_double[2], kernel_size=(7, 1), strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_7x7_double)
  branch_7x7_double = keras.layers.BatchNormalization()(branch_7x7_double)
  branch_7x7_double = keras.layers.ReLU()(branch_7x7_double)
  # a 1x7 conv layer
  branch_7x7_double = keras.layers.Conv2D(filters=filters_7x7_double[3], kernel_size=(1, 7), strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_7x7_double)
  branch_7x7_double = keras.layers.BatchNormalization()(branch_7x7_double)
  branch_7x7_double = keras.layers.ReLU()(branch_7x7_double)
  # a 7x1 conv layer
  branch_7x7_double = keras.layers.Conv2D(filters=filters_7x7_double[4], kernel_size=(7, 1), strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_7x7_double)
  branch_7x7_double = keras.layers.BatchNormalization()(branch_7x7_double)
  branch_7x7_double = keras.layers.ReLU()(branch_7x7_double)

  # build the pooling branch with average pooling and a 1x1 conv layer
  branch_pool = keras.layers.AveragePooling2D(pool_size=3, strides=1, padding='same')(x)
  branch_pool = keras.layers.Conv2D(filters=filters_pool[0], kernel_size=1, strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_pool)
  branch_pool = keras.layers.BatchNormalization()(branch_pool)
  branch_pool = keras.layers.ReLU()(branch_pool)

  # Concatenate the outputs (filters) of the branches
  x = keras.layers.Concatenate()([branch_1x1, branch_7x7, branch_7x7_double, branch_pool])

  return x

### Inception Block C

In [23]:
def inception_block_C(x, filters_1x1, filters_3x3, filters_3x3_double, filters_pool):
  """ Build an Inception block 8x8.
      x                 : input to the block
      filters_1x1       : filters for 1x1 branch
      filters_3x3       : filters for 3x3 factorized asn 1x3, 3x1 branch
      filters_3x3_double: filters for double 3x3 factorized as 1x3, 3x1 branch
      filters_pool      : filters for pooling branch
  """
  # build the 1x1 branch
  branch_1x1 = keras.layers.Conv2D(filters=filters_1x1[0], kernel_size=1, strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(x)
  branch_1x1 = keras.layers.BatchNormalization()(branch_1x1)
  branch_1x1 = keras.layers.ReLU()(branch_1x1)

  # build the 3x3 branch with a 1x1 conv layer
  branch_3x3 = keras.layers.Conv2D(filters=filters_3x3[0], kernel_size=1, strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(x)
  branch_3x3 = keras.layers.BatchNormalization()(branch_3x3)
  branch_3x3 = keras.layers.ReLU()(branch_3x3)
  # split the branch into a 1x3 conv layer and a 3x1 conv layer
  branch_3x3_1 = keras.layers.Conv2D(filters=filters_3x3[0], kernel_size=(1, 3), strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_3x3)
  branch_3x3_1 = keras.layers.BatchNormalization()(branch_3x3_1)
  branch_3x3_1 = keras.layers.ReLU()(branch_3x3_1)
  branch_3x3_2 = keras.layers.Conv2D(filters=filters_3x3[1], kernel_size=(3, 1), strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_3x3)
  branch_3x3_2 = keras.layers.BatchNormalization()(branch_3x3_2)
  branch_3x3_2 = keras.layers.ReLU()(branch_3x3_2)
  # merge the two sub-branches
  branch_3x3   = keras.layers.Concatenate()([branch_3x3_1, branch_3x3_2])

  # build a double 3x3 branch with a 1x1 conv layer
  branch_3x3_double = keras.layers.Conv2D(filters=filters_3x3_double[0], kernel_size=1, strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(x)
  branch_3x3_double = keras.layers.BatchNormalization()(branch_3x3_double)
  branch_3x3_double = keras.layers.ReLU()(branch_3x3_double)
  branch_3x3_double = keras.layers.Conv2D(filters=filters_3x3_double[1], kernel_size=3, strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_3x3_double)
  branch_3x3_double = keras.layers.BatchNormalization()(branch_3x3_double)
  branch_3x3_double = keras.layers.ReLU()(branch_3x3_double)
  # split the branch into a 1x3 conv layer and 3x1 conv layer
  branch_3x3_double_1 = keras.layers.Conv2D(filters_3x3_double[2], (1, 3), strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_3x3_double)
  branch_3x3_double_1 = keras.layers.BatchNormalization()(branch_3x3_double_1)
  branch_3x3_double_1 = keras.layers.ReLU()(branch_3x3_double_1)
  branch_3x3_double_2 = keras.layers.Conv2D(filters_3x3_double[3], (3, 1), strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_3x3_double)
  branch_3x3_double_2 = keras.layers.BatchNormalization()(branch_3x3_double_2)
  branch_3x3_double_2 = keras.layers.ReLU()(branch_3x3_double_2)
  # merge the two sub-branches
  b3x3dbl   = keras.layers.Concatenate()([branch_3x3_double_1, branch_3x3_double_2])

  # build the pooling branch with average pooling and a 1x1 conv layer
  branch_pool = keras.layers.AveragePooling2D(pool_size=3, strides=1, padding='same')(x)
  branch_pool = keras.layers.Conv2D(filters=filters_pool[0], kernel_size=(1, 1), strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_pool)
  branch_pool = keras.layers.BatchNormalization()(branch_pool)
  branch_pool = keras.layers.ReLU()(branch_pool)

  # concatenate the outputs (filters) of the branches
  x = keras.layers.Concatenate()([branch_1x1, branch_3x3, branch_3x3_double, branch_pool])

  return x

### Grid Reduction A

In [24]:
def grid_reduction_A(x, filters_3x3=384, filters_3x3_double=(64, 96, 96)):
  """ Build the Grid Reduction block
      x                   : input to the block
      filters_3x3         : filter size for 3x3 branch
      filters_3x3_double  : filter sizes for double 3x3 branch
  """
  # build the 3x3 branch for grid reduction
  branch_3x3 = keras.layers.Conv2D(filters=filters_3x3, kernel_size=3, strides=2, padding='valid', use_bias=False, kernel_initializer='glorot_uniform')(x)
  branch_3x3 = keras.layers.BatchNormalization()(branch_3x3)
  branch_3x3 = keras.layers.ReLU()(branch_3x3)

  # build the double 3x3 branch with a 1x1 conv layer for dimensionality reduction
  branch_3x3_double = keras.layers.Conv2D(filters=filters_3x3_double[0], kernel_size=1, strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(x)
  branch_3x3_double = keras.layers.BatchNormalization()(branch_3x3_double)
  branch_3x3_double = keras.layers.ReLU()(branch_3x3_double)
  branch_3x3_double = keras.layers.Conv2D(filters=filters_3x3_double[1], kernel_size=(3, 3), strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_3x3_double)
  branch_3x3_double = keras.layers.BatchNormalization()(branch_3x3_double)
  branch_3x3_double = keras.layers.ReLU()(branch_3x3_double)
  # a 3x3 conv layer for grid reduction
  branch_3x3_double = keras.layers.Conv2D(filters=filters_3x3_double[1], kernel_size=3, strides=2, padding='valid', use_bias=False, kernel_initializer='glorot_uniform')(branch_3x3_double)
  branch_3x3_double = keras.layers.BatchNormalization()(branch_3x3_double)
  branch_3x3_double = keras.layers.ReLU()(branch_3x3_double)

  # pool branch
  branch_pool = keras.layers.MaxPooling2D(pool_size=3, strides=2)(x)

  # Concatenate the outputs (filters) of the branches
  x = keras.layers.Concatenate()([branch_3x3, branch_3x3_double, branch_pool])

  return x

### Grid Reduction B

In [25]:
def grid_reduction_B(x, filters_3x3=(192, 320), filters_7x7=(192, 192, 192, 192)):
  """ Build the Grid Reduction block
      x               : input to the block
      filters_3x3     : filter size for 3x3 branch
      filters_7x7     : filter sizes for 7x7 + 3x3 branch
  """
  # build the 3x3 branch with a 1x1 conv layer
  branch_3x3 = keras.layers.Conv2D(filters=filters_3x3[0], kernel_size=1, strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(x)
  branch_3x3 = keras.layers.BatchNormalization()(branch_3x3)
  branch_3x3 = keras.layers.ReLU()(branch_3x3)
  # build the grid reduction
  branch_3x3 = keras.layers.Conv2D(filters=filters_3x3[1], kernel_size=3, strides=2, padding='valid', use_bias=False, kernel_initializer='glorot_uniform')(branch_3x3)
  branch_3x3 = keras.layers.BatchNormalization()(branch_3x3)
  branch_3x3 = keras.layers.ReLU()(branch_3x3)

  # build the 7x7 conv layer (factorized as 1x7, 7x1) with the 1x1 conv layer for dimensionality reduction
  branch_7x7 = keras.layers.Conv2D(filters=filters_7x7[0], kernel_size=1, strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(x)
  branch_7x7 = keras.layers.BatchNormalization()(branch_7x7)
  branch_7x7 = keras.layers.ReLU()(branch_7x7)
  # build the 1x7 conv layer
  branch_7x7 = keras.layers.Conv2D(filters=filters_7x7[1], kernel_size=(1, 7), strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_7x7)
  branch_7x7 = keras.layers.BatchNormalization()(branch_7x7)
  branch_7x7 = keras.layers.ReLU()(branch_7x7)
  branch_7x7 = keras.layers.Conv2D(filters=filters_7x7[2], kernel_size=(7, 1), strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(branch_7x7)
  branch_7x7 = keras.layers.BatchNormalization()(branch_7x7)
  branch_7x7 = keras.layers.ReLU()(branch_7x7)
  # grid reduction
  branch_7x7 = keras.layers.Conv2D(filters=filters_7x7[3], kernel_size= 3, strides=2, padding='valid', use_bias=False, kernel_initializer='glorot_uniform')(branch_7x7)
  branch_7x7 = keras.layers.BatchNormalization()(branch_7x7)
  branch_7x7 = keras.layers.ReLU()(branch_7x7)

  # build a pool branch
  branch_pool = keras.layers.MaxPooling2D(pool_size=3, strides=2)(x)

  # concatenate the outputs (filters) of the branches
  x = keras.layers.Concatenate()([branch_3x3, branch_7x7, branch_pool])

  return x

### Putting it all together

In [26]:
def learner(x, classes):
  """ Build the Learner
      x         : input to the learner
      classes   : number of output classes
  """
  aux_outputs = [] # Auxiliary Outputs

  # Group A (35x35)
  x, o = group(x, [((64,), (64, 96), (48, 64), (32,)),
                   ((64,), (64, 96), (48, 64), (64,)),
                   ((64,), (64, 96), (48, 64), (64,))
                  ], inception=inception_block_A, reduction=grid_reduction_A)
  aux_outputs += o

  # Group B (17x17)
  x, o = group(x, [((192,), (128, 128, 192), (128, 128, 128, 128, 192), (192,)),
                   ((192,), (160, 160, 192), (160, 160, 160, 160, 192), (192,)),
                   ((192,), (160, 160, 192), (160, 160, 160, 160, 192), (192,)),
                   ((192,), (192, 192, 192), (192, 192, 192, 192, 192), (192,))
                   ], inception=inception_block_B, reduction=grid_reduction_B, classes=classes)
  aux_outputs += o

  # Group C (8x8)
  x, o = group(x, [((320,), (384, 384, 384), (448, 384, 384, 384), (192,)),
                   ((320,), (384, 384, 384), (448, 384, 384, 384), (192,))
                  ], inception=inception_block_C)
  aux_outputs += o
  return x, aux_outputs

## The task component

In [27]:
def task(x, classes, dropout=0.4):
  """ Build the task.
      x         : input to the classifier
      classes   : number of output classes
      dropout   : percentage for dropout rate
  """
  # pool at the end of all the convolutional residual blocks will be 8x8 in V3
  x = keras.layers.AveragePooling2D(pool_size=x.shape[1:3].as_list())(x)
  x = keras.layers.Dropout(dropout)(x)
  x = keras.layers.Flatten()(x)

  # outputs
  outputs = keras.layers.Dense(units=classes, activation='softmax', kernel_initializer='glorot_uniform')(x)


  return outputs

## Inception model v3

In [28]:
def model():
  # The input tensor (299x299 in V3 vs 224x224 in V1/V2)
  inputs = keras.Input(shape=(299, 299, 3))

  # step
  x = stem(inputs)

  # The learner
  x, aux = learner(x, 1000)

  # The classifier for 1000 classes
  outputs = task(x, 1000)

  # Instantiate and return the Model
  return keras.Model(inputs, [outputs] + aux, name="Inception-v3")

In [29]:
# create the inception model (v3)
inception_model_v3 = model()

# print a summary
inception_model_v3.summary()

Model: "Inception-v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d_118 (Conv2D)         (None, 149, 149, 32)         864       ['input_3[0][0]']             
                                                                                                  
 batch_normalization_59 (Ba  (None, 149, 149, 32)         128       ['conv2d_118[0][0]']          
 tchNormalization)                                                                                
                                                                                                  
 re_lu_59 (ReLU)             (None, 149, 149, 32)         0         ['batch_normalizati